# Deep Learning
本章では、点群に対する深層学習手法について紹介します。Deep learning is a method of learning optimal features from given data. This method has some limitations, such as the need to prepare data that matches the target task, but if the limitations can be met, it has the potential to achieve better results than other methods. For this reason, methods using this deep learning approach have been actively researched in recent years. 
点群に対する深層学習手法はクラス分類やセグメンテーション等様々なタスクに対して提案されています。以下の様に、これらの点群の深層学習手法の根幹は、画像に対する深層学習手法等と似ていることが多いです。ただし、画像等と比べて点群が決まった構造を持たない表現であるため、前処理や深層学習のモデルが点群特有である場合があります。本章では、点群深層学習で一般的なタスクであるクラス分類タスクに基づいた深層学習モデルの紹介を行います。

本章では、代表的で実装が比較的簡単な手法を紹介します。紹介する手法は以下の通りです。

- VoxNet
- PointNet
- PointNet++

In [1]:
%load_ext autoreload
%autoreload 2

## VoxNet
VoxNetは占有モデルを入力とするネットワークです。VoxNetでは点群を占有モデルへ変換し、その占有モデルに基づいてクラス分類を行うことができます。この手法の利点は以下の通りです。

- 3DCNNsの利用: グリッドに沿った表現である占有モデルを利用するため、2DCNNsを拡張した3DCNNsを点群等の立体的な表現へ適用することが可能となります。これにより、2DCNNsの手法の応用が用意となります。
- 効率的な計算: グリッドに沿って近傍情報の畳み込みを行うことが可能となる。グリッドに沿った畳み込みを行わない場合、kNN等を利用して近い点を探す必要があり、処理時間が増える可能性が高い。ダウンサンプリング処理もグリッドに沿って効率的に可能である。

In [2]:
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from torch import optim
from torch import nn

from tutlibs.dl.PointNet import PointNetClassification
from tutlibs.dl.dataset import ModelNet40Dataset
from tutlibs.dl.loss import feature_transform_regularizer


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
def main():
    epochs = 250
    device = 0

    dataset = ModelNet40Dataset("../data/modelnet40_ply_hdf5_2048/")

    model = PointNetClassification(40)
    model = model.to(device=device)

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    scheduler = optim.lr_scheduler.StepLR(
        optimizer,
        step_size=20,
        gamma=0.7,
    )

    loss_ce = nn.CrossEntropyLoss()
    loss_ftr = feature_transform_regularizer

    for epoch in range(epochs):
        print("Epoch {}/{}:".format(epoch, epochs))
        loader = DataLoader(dataset, 32, shuffle=True)
        model.train()
        for data in tqdm(loader, desc="batch", ncols=60):
            optimizer.zero_grad()

            point_clouds, gt_labels = data
            point_clouds = torch.transpose(point_clouds, 1, 2).to(
                device, dtype=torch.float32
            )
            gt_labels = gt_labels.to(device, dtype=torch.long)

            pred_labels, _, feature_transformation_matrix = model(point_clouds)

            loss = loss_ce(pred_labels, gt_labels)
            if feature_transformation_matrix is not None:
                loss += loss_ftr(feature_transformation_matrix) * 0.001

            loss.backward()
            optimizer.step()

        scheduler.step()

    torch.save(
        {
            "epoch": epoch,
            "model": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "scheduler": scheduler.state_dict(),
        },
        "model_path.pth",
    )


## PointNet
PointNet[Qi et al. 2017a]は点群から点ごとの特徴(local (pointwise?) feature)と点群全体の特徴(global feature)を抽出することが可能なネットワークです。点群を適切に処理可能にするため、以下の問題に対して以下の構造を提案した(点群の問題については[characteristic.ipynb](characteristic.ipynb))。

- 点の順不同性: Point-wise convolution layerを採用することで個々の点が持つ点のみを畳み込む。Poolingは点方向に行う((N, 1024) -> (1024))。
- オブジェクトのランダム向き: TransformationNetworkを入力点群と特徴量に対して設けた。

PointNetアーキテクチャは以下の通り。

![pointnet](img/pointnet.png)

PointNetを用いた推論は以下の通り。

In [2]:
import os

from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from torch import optim
from torch import nn

from tutlibs.dl.PointNet import PointNetClassification
from tutlibs.dl.dataset import (
    ModelNet40Dataset,
    rotate_point_cloud,
    jitter_point_cloud,
)
from tutlibs.dl.loss import feature_transform_regularizer
from tutlibs.dl.utils import t2n


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [6]:
def test():
    device = 0
    output_dir_path = "outputs/PointNet/"
    dataset_dir_path = "../data/modelnet40_ply_hdf5_2048/"
    num_points = 1024
    num_classes = 40

    os.makedirs(output_dir_path, exist_ok=True)

    dataset = ModelNet40Dataset(dataset_dir_path, mode="test")

    model = PointNetClassification(40)
    model = model.to(device=device)
    checkpoint = torch.load(os.path.join(output_dir_path, "model_path.pth"))
    model.load_state_dict(checkpoint["model"])

    loader = DataLoader(dataset, 32, shuffle=False)
    model.eval()

    results = []

    with torch.no_grad():
        for data in tqdm(loader, desc="test", ncols=60):
            point_clouds, gt_labels = data

            point_clouds = point_clouds[:, 0:num_classes]
            point_clouds = torch.transpose(point_clouds, 1, 2).to(
                device, dtype=torch.float32
            )
            gt_labels = gt_labels.to(device, dtype=torch.long)

            net_output, _, _ = model(point_clouds)
            pred_labels = torch.argmax(net_output, dim=1)
            results.append(pred_labels == gt_labels)

    results = torch.cat(results, dim=0)
    acc = torch.sum(results) / len(results) * 100
    print(f"accuracy: {acc}")


test()


test: 100%|████████████████| 78/78 [00:00<00:00, 298.01it/s]

accuracy: 25.85089111328125


訓練は以下の通り。

In [4]:
def train():
    epochs = 250
    device = 0
    output_dir_path = "outputs/PointNet"
    dataset_dir_path = "../data/modelnet40_ply_hdf5_2048/"
    num_points = 1024
    num_classes = 40

    os.makedirs(output_dir_path, exist_ok=True)

    dataset = ModelNet40Dataset(dataset_dir_path, mode="train")

    model = PointNetClassification(num_classes)
    model = model.to(device=device)

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    scheduler = optim.lr_scheduler.StepLR(
        optimizer,
        step_size=20,
        gamma=0.7,
    )

    loss_ce = nn.CrossEntropyLoss()
    loss_ftr = feature_transform_regularizer

    for epoch in tqdm(range(epochs), desc="training epoch", ncols=60):
        loader = DataLoader(dataset, 32, shuffle=True)
        model.train()
        for data in loader:
            optimizer.zero_grad()

            point_clouds, gt_labels = data

            point_clouds = point_clouds[:, 0:num_points]
            point_clouds = rotate_point_cloud(t2n(point_clouds))
            point_clouds = jitter_point_cloud(point_clouds)

            point_clouds = torch.tensor(
                point_clouds, dtype=torch.float32, device=device
            ).transpose(1, 2)
            gt_labels = gt_labels.to(dtype=torch.long, device=device)

            net_output, _, feature_transformation_matrix = model(point_clouds)

            loss = loss_ce(net_output, gt_labels)
            if feature_transformation_matrix is not None:
                loss += loss_ftr(feature_transformation_matrix) * 0.001

            loss.backward()
            optimizer.step()

        scheduler.step()

    torch.save(
        {
            "epoch": epoch,
            "model": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "scheduler": scheduler.state_dict(),
        },
        os.path.join(output_dir_path, "model_path.pth"),
    )


train()


training epoch: 100%|█████| 250/250 [47:25<00:00, 11.38s/it]


## PointNet++
PointNet++[Qi et al. 2017b]は、点の局所領域の特徴を抽出する機構を持つネットワークです。PointNetでは、点ごとの特徴のみを畳みこんでいましたが、本提案ではhybrid searchを利用して点ごとに近傍点を求め、その近傍点間の関係性を畳みこむ機構を持ちます。本提案はPointNetと比較した利点が以下の通りです。

- 局所領域の畳み込み: hybrid searchを用いて近傍点とそのクエリの相対位置を用いたプーリングと畳み込みを行います。handcrafted feature等でも利用されていたように、局所領域の関係性は点ごとの識別的な特徴を持つうえで重要とされています。

In [ ]:
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from torch import optim
from torch import nn

from tutlibs.dl.PointNet import PointNetClassification
from tutlibs.dl.dataset import ModelNet40Dataset
from tutlibs.dl.loss import feature_transform_regularizer


In [ ]:
def main():
    epochs = 250
    device = 0

    dataset = ModelNet40Dataset("../data/modelnet40_ply_hdf5_2048/")

    model = PointNetClassification(40)
    model = model.to(device=device)

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    scheduler = optim.lr_scheduler.StepLR(
        optimizer,
        step_size=20,
        gamma=0.7,
    )

    loss_ce = nn.CrossEntropyLoss()
    loss_ftr = feature_transform_regularizer

    for epoch in range(epochs):
        print("Epoch {}/{}:".format(epoch, epochs))
        loader = DataLoader(dataset, 32, shuffle=True)
        model.train()
        for data in tqdm(loader, desc="batch", ncols=60):
            optimizer.zero_grad()

            point_clouds, gt_labels = data
            point_clouds = torch.transpose(point_clouds, 1, 2).to(
                device, dtype=torch.float32
            )
            gt_labels = gt_labels.to(device, dtype=torch.long)

            pred_labels, _, feature_transformation_matrix = model(point_clouds)

            loss = loss_ce(pred_labels, gt_labels)
            if feature_transformation_matrix is not None:
                loss += loss_ftr(feature_transformation_matrix) * 0.001

            loss.backward()
            optimizer.step()

        scheduler.step()

    torch.save(
        {
            "epoch": epoch,
            "model": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "scheduler": scheduler.state_dict(),
        },
        "model_path.pth",
    )


## References
- Maturana, Daniel, and Sebastian Scherer. 2015. “VoxNet: A 3D Convolutional Neural Network for Real-Time Object Recognition.” In 2015 IEEE/RSJ International Conference on Intelligent Robots and Systems (IROS). IEEE. https://doi.org/10.1109/iros.2015.7353481.
- Qi, Charles R., Hao Su, Kaichun Mo, and Leonidas J. Guibas. 2017. “Pointnet: Deep Learning on Point Sets for 3d Classification and Segmentation.” In Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition, 652–60.
- Qi, Charles R., Li Yi, Hao Su, and Leonidas J. Guibas. 2017. “PointNet++: Deep Hierarchical Feature Learning on Point Sets in a Metric Space.” arXiv [cs.CV]. arXiv. http://arxiv.org/abs/1706.02413.